# Cruise Control Module & Vehicle Model
## DARPA Assured Micro-Patching Project
## Peter Lobato, Matt Campo, Ben Ettlinger
### Version 0-Beaglebone

This jupyter notebook contains two parts: (a) a simulated cruise control module (CCM) and (b) a vehicle model (VM).

The CCM takes in vehicle speed and desired setpoint and outputs an accelerator pedal position (APP). The VM takes in APP and outputs an incremental change in vehicle speed. This interaction becomes a loop which runs at 10Hz. 

The vehicle used in this model is a Class 6 Freightliner truck with road load coefficients found here: 

https://www.nrel.gov/docs/fy17osti/64610.pdf

The cruise control module settings can be configured by the user. These include the Kp and Ki terms, (+) and (-) slew rates, speed differential at which the CCM shuts off, and integrator time constant.

Notes:

APP and APP_Tx are intentionally separated. APP_Tx is the value written to an outgoing CAN message. APP is a number used within Jupyter. They are separate because the vehicle model should read APP from the CAN bus and not a variable within Python. The same is true for kph and kph_Tx.


### Version Control

Version 0: First version made on Windows PC as a mock-up

Version 0-BeagleBone: Same as version 0, but with matplotlib functions removed. Used on the Beaglebone/debian


In [1]:
# Import libraries
import time
import math
import numpy as np
import socket
import struct
import sys
#import git
#g = git.cmd.Git(plobato3/hello-world)
#import matplotlib.pyplot as plt
# import pandas as pd
# import xlrd
# pip install numpy==1.19.3

In [2]:
# Define User Variables
#Global Variables
t_interval = .1  # Constant time interval [reflects CAN protocol wanted]

In [3]:
#User-defined variables for CCM
initial_kph = 55  # Initial speed when Cruise Control is activated [kph]
target = 75.0  # Speed wanted to be maintained [kph]
kff = 0
c_kp = 10  # Constant proportional term
c_ki = 0.01  # Constant integral term
itc = 100  # Integral time constant [used per time interval, units are number of time intervals, or 1/10th seconds]
slew_lowLimit = -75  # Low limit for calculated slew rate [%/s]
slew_highLimit = 75
# High limit for calculated slew rate [%/s]
APP = 0  # Initial accelerator pedal position [%]
APP_Tx = 0
prev_APP = 0
kph = initial_kph
prev_kph = initial_kph
prev_errors = [0.0 for _ in range(itc)]  # Array of previous errors to calculate integral term
i = 0  # Keeps circular buffer of size itc for error summation
checks = True  # Breaks proccess if errors occur
graph = True
# Using to check results via graph
graph = True
x = [0.0]
y = [initial_kph]
x_value = 0.0
y_value = initial_kph

In [4]:
#This cell simulates the cruise control module receiving speed from the CAN bus
def CAN_kph_Rx():
    """Documentation: This function reads a single CAN message if it has ArbID 0x18FEF125 for vehicle speed."""
    
    global kph
    
    speed_m = 1/256 #Multiplier for speed from J1939 standard [counts/kph]
    speed_b = 0 #Offset for speed from J1939 [kph]
    
    # Open a socket and bind to it from SocketCAN
    sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
    sock.setsockopt(socket.SOL_CAN_RAW, socket.CAN_RAW_ERR_FILTER, socket.CAN_ERR_MASK) #allows socket to receive error frames
    # The interface can be seen from the command prompt (ifconfig)
    # The can channel must be configured using the ip link commands
    interface = "vcan0"

    # Bind to the interface
    try:
        sock.bind((interface,))
    except OSError:
        print("Could not bind to interface '%s'\n" % interface)
        sys.exit() 

    # To match this data structure, the following struct format can be used:
    can_frame_format = "<LB3x8s"
    # Unsigned Long Integer (little endian), unsigned char (byte), three pad bytes, eight chars (bytes)
    # Note: this is 16 bytes


    # These are defined in can.h of the Linux kernel sources
    # e.g. https://github.com/torvalds/linux/blob/master/include/uapi/linux/can.h
    print("socket.CAN_EFF_MASK = 0x{:08X}".format(socket.CAN_EFF_MASK))
    print("socket.CAN_EFF_FLAG = 0x{:08X}".format(socket.CAN_EFF_FLAG))
    print("socket.CAN_RTR_FLAG = 0x{:08X}".format(socket.CAN_RTR_FLAG))
    print("socket.CAN_ERR_FLAG = 0x{:08X}".format(socket.CAN_ERR_FLAG))

    # Enter a loop to read and display the data
    while True:
        can_packet = sock.recv(16) 
        can_id, can_dlc, can_data = struct.unpack(can_frame_format, can_packet)

        if (can_id == 2566844709): #ArbID for Speed. Change for APP
            break
    extended_frame = bool(can_id & socket.CAN_EFF_FLAG)
    error_frame = bool(can_id & socket.CAN_ERR_FLAG)
    remote_tx_req_frame = bool(can_id & socket.CAN_RTR_FLAG)

    if error_frame:
        can_id &= socket.CAN_ERR_MASK
        can_id_string = "{:08X} (ERROR)".format(can_id)
    else: #Data Frame
        if extended_frame:
            can_id &= socket.CAN_EFF_MASK
            can_id_string = "{:08X}".format(can_id)
        else: #Standard Frame
            can_id &= socket.CAN_SFF_MASK
            can_id_string = "{:03X}".format(can_id)

    if remote_tx_req_frame:
        can_id_string = "{:08X} (RTR)".format(can_id)

    hex_data_print = ' '.join(["{:02X}".format(can_byte) for can_byte in can_data])
    print("{:12.6f} {} [{}] {}".format(time.time(), can_id_string, can_dlc, hex_data_print))
    kph = int(hex_data_print[18:20] + hex_data_print[15:17],base=16)*speed_m + speed_b

    return kph #Units [kph]


In [ ]:
#This cell simulates the cruise control module transmitting APP onto the CAN bus
def CAN_APP_Tx(APP_Tx):
    """Documentation: This function takes in the value 'Driver's Demand Engine - Percent Torque' as a
    floating-point number (type='float'), turns it into the J1939 CAN message format, and sends the message
    on bus 'VCAN0' using SocketCAN. It uses PGN 61444, SPN 512. Arbitration is set to 18. These can be changed"""

    APP_m = 1 #Multiplier for APP from J1939 standard [counts/%]
    APP_b = -125 #Offset for APP from J1939 [%]

    app_counts = int((APP - APP_b) / APP_m) #turns floating-point value from Python into base 10 integer number
    app_hex = hex(app_counts)
    app_byte1 = int(app_hex[2] + app_hex[3],16) #turns into a base-16 integer number
    #app_byte2 = int(app_hex[4] + app_hex[5],16)

    print("APP: ",APP)
    print("app_counts: ",app_counts)
    print("app_hex: ",app_hex)
    print("app_byte1: ",app_byte1)
    #print("app_byte2: ",app_byte2)
    
    # Open a socket and bind to it from SocketCAN
    sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)

    #Change this interface to match your desired connection
    interface = "vcan0"

    # Bind to the interface
    sock.bind((interface,))

    can_frame_format = "=lB3x8s"

    # Define the CAN Arbitration ID
    can_id = 0x18F00326 #For Driver's Demand on PGN 61444. Source Address = 26 (for no real reason)
    #Set the extended frame format bit.
    can_id |= socket.CAN_EFF_FLAG
    
    can_data = struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",app_byte1) + struct.pack("B",0xFF)
    #The data length code cannot be larger than 8
    can_dlc = min(len(can_data),8)
    
    #Pack the information into 16 bytes
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
    #can_packet = struct.pack(can_frame_format, can_id, can_dlc, kph_Tx)

    print(can_packet)
    
    #Send out the CAN frame
    sock.send(can_packet)
    
    pass


In [ ]:
#Cruise Controller
def cruise_controller(kph):
    global i
    global prev_APP
    global APP
    global ki
    global kp

    error = kph - target
    prev_errors[i] = error
    i = i + 1
    if i == itc:
        i = 0
    kp = -c_kp * error
    ki = -c_ki * sum(prev_errors)
    APP_Tx = kp + ki

    if APP_Tx > 100:
        APP_Tx = 100
    elif APP_Tx < 0:
        APP_Tx = 0

    slew_rate = (APP - prev_APP) / t_interval

    if slew_rate < slew_lowLimit:
        slew_rate = slew_lowLimit
        APP_Tx = prev_APP + slew_rate * t_interval
    elif slew_rate > slew_highLimit:
        slew_rate = slew_highLimit
        APP_Tx = prev_APP + slew_rate * t_interval

    prev_APP = APP
    APP = APP_Tx
    return APP_Tx


In [ ]:
# Main Response Loop: Cruise Control Module

while (checks == True):

    # Error Handlers
    if abs(target - kph) > 30:
        print("ERROR: Target speed outside speed difference limit")
        checks = False
        break
        
    # Main Loop
    CAN_kph_Rx()
    cruise_controller(kph)
    CAN_APP_Tx(APP_Tx)
    time.sleep(0.1)
    #CAN_APP_Rx()
    #vehicle_model(APP)
    #CAN_kph_TX(kph_Tx)
    #time.sleep(0.05)

    # Generated Graph of Response
    if graph == True:
        x_value+=t_interval
        y_value = kph
        #g_value = grade
        x.append(x_value)
        y.append(y_value)
        #gout.append(g_value)
        print(x_value)#Just put in to make sure program is running properly
        print(y_value) #Uncomment these three "print" statements to show numerical values
        #Use this stuff if running in a PC and can use matplotlib
        if x_value >= 100: #Change this number for how long you want loop to run
            graph = False
            checks = False
            #plt.plot(x,y)
            #plt.plot(x,y2)
            #plt.xlim(0,60)
            #plt.ylim(0,110)
            #plt.grid(True)
            #plt.xlabel('Time [s]')
            #plt.ylabel('Speed [kph]')
            #plt.title('Cruise Control Response')
            #plt.show()
